# XGBoost Workbook
splitting it out cause it's more complicated

this works with the 0.72 but not 0.9

In [2]:
!wget https://repo1.maven.org/maven2/ml/dmlc/xgboost4j/0.72/xgboost4j-0.72.jar -O ../data/libs/xgboost4j-0.72.jar
!wget https://repo1.maven.org/maven2/ml/dmlc/xgboost4j-spark/0.72/xgboost4j-spark-0.72.jar -O ../data/libs/xgboost4j-spark-0.72.jar
#!wget https://github.com/dmlc/xgboost/files/2161553/sparkxgb.zip -O ../data/libs/sparkxgb.zip

--2020-04-13 23:34:34--  https://repo1.maven.org/maven2/ml/dmlc/xgboost4j/0.72/xgboost4j-0.72.jar
Resolving repo1.maven.org (repo1.maven.org)... 151.101.40.209
Connecting to repo1.maven.org (repo1.maven.org)|151.101.40.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2134592 (2.0M) [application/java-archive]
Saving to: ‘../data/libs/xgboost4j-0.72.jar’

../data/libs/xgboos 100%[===================>]   2.04M  1.21MB/s    in 1.7s    

2020-04-13 23:34:37 (1.21 MB/s) - ‘../data/libs/xgboost4j-0.72.jar’ saved [2134592/2134592]

--2020-04-13 23:34:37--  https://repo1.maven.org/maven2/ml/dmlc/xgboost4j-spark/0.72/xgboost4j-spark-0.72.jar
Resolving repo1.maven.org (repo1.maven.org)... 151.101.40.209
Connecting to repo1.maven.org (repo1.maven.org)|151.101.40.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 296184 (289K) [application/java-archive]
Saving to: ‘../data/libs/xgboost4j-spark-0.72.jar’

../data/libs/xgboos 100%[==============

In [3]:
!wget https://repo1.maven.org/maven2/ml/dmlc/xgboost4j/0.90/xgboost4j-0.90.jar -O /opt/spark-data/libs/xgboost4j-0.90.jar
!wget https://repo1.maven.org/maven2/ml/dmlc/xgboost4j-spark/0.90/xgboost4j-spark-0.90.jar -O /opt/spark-data/libs/xgboost4j-spark-0.90.jar

/opt/spark-data/libs/xgboost4j-0.90.jar: Permission denied
/opt/spark-data/libs/xgboost4j-spark-0.90.jar: Permission denied


In [4]:
!wget https://github.com/dmlc/xgboost/files/3384356/pyspark-xgboost_0.90_261ab52e07bec461c711d209b70428ab481db470.zip -O /opt/spark-data/libs/pyspark-xgboost.zip

/opt/spark-data/libs/pyspark-xgboost.zip: Permission denied


In [5]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /opt/spark-data/libs/xgboost4j-spark-0.90.jar,/opt/spark-data/libs/xgboost4j-0.90.jar pyspark-shell'

In [6]:
from pyspark.sql import SparkSession

In [7]:
spark = SparkSession\
        .builder\
        .appName("PySpark xgb new")\
        .master("spark://spark-master:7077")\
        .getOrCreate()

In [8]:
!ls /opt/spark-data/libs

pyspark-xgboost.zip  xgboost4j-0.90.jar  xgboost4j-spark-0.90.jar


In [9]:
spark.sparkContext.addPyFile("/opt/spark-data/libs/pyspark-xgboost.zip")

In [10]:
dataset = spark.read.format("libsvm").load("/opt/spark-data/iris.scale")
# split dataset into train and test
train, test = dataset.randomSplit([0.8,0.2])

In [11]:
dataset.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



lets look at one entry

In [12]:
type(dataset.select("features")) #.toDense()

pyspark.sql.dataframe.DataFrame

In [13]:
dataset.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  1.0|(4,[0,1,2,3],[-0....|
|  1.0|(4,[0,1,2,3],[-0....|
|  1.0|(4,[0,2,3],[-0.77...|
|  1.0|(4,[0,1,2,3],[-0....|
|  1.0|(4,[0,1,2,3],[-0....|
|  1.0|(4,[0,1,2,3],[-0....|
|  1.0|(4,[0,1,2,3],[-0....|
|  1.0|(4,[0,1,2,3],[-0....|
|  1.0|(4,[0,1,2,3],[-0....|
|  1.0|(4,[0,1,2,3],[-0....|
|  1.0|(4,[0,1,2,3],[-0....|
|  1.0|(4,[0,1,2,3],[-0....|
|  1.0|(4,[0,1,2,3],[-0....|
|  1.0|(4,[0,1,2,3],[-1....|
|  1.0|(4,[0,1,2,3],[-0....|
|  1.0|(4,[0,1,2,3],[-0....|
|  1.0|(4,[0,1,2,3],[-0....|
|  1.0|(4,[0,1,2,3],[-0....|
|  1.0|(4,[0,1,2,3],[-0....|
|  1.0|(4,[0,1,2,3],[-0....|
+-----+--------------------+
only showing top 20 rows



In [14]:
type(dataset)

pyspark.sql.dataframe.DataFrame

In [15]:
from pyspark.sql.types import *
schema = StructType([
            StructField("sepal length", DoubleType(), True),
            StructField("sepal width", DoubleType(), True),
            StructField("petal length", DoubleType(), True),
            StructField("petal width", DoubleType(), True),
            StructField("class", StringType(), True)
        ])

In [16]:
rawInput = spark.read.schema(schema).csv("/opt/spark-data/iris.data")

In [17]:
rawInput.show()

+------------+-----------+------------+-----------+-----------+
|sepal length|sepal width|petal length|petal width|      class|
+------------+-----------+------------+-----------+-----------+
|         5.1|        3.5|         1.4|        0.2|Iris-setosa|
|         4.9|        3.0|         1.4|        0.2|Iris-setosa|
|         4.7|        3.2|         1.3|        0.2|Iris-setosa|
|         4.6|        3.1|         1.5|        0.2|Iris-setosa|
|         5.0|        3.6|         1.4|        0.2|Iris-setosa|
|         5.4|        3.9|         1.7|        0.4|Iris-setosa|
|         4.6|        3.4|         1.4|        0.3|Iris-setosa|
|         5.0|        3.4|         1.5|        0.2|Iris-setosa|
|         4.4|        2.9|         1.4|        0.2|Iris-setosa|
|         4.9|        3.1|         1.5|        0.1|Iris-setosa|
|         5.4|        3.7|         1.5|        0.2|Iris-setosa|
|         4.8|        3.4|         1.6|        0.2|Iris-setosa|
|         4.8|        3.0|         1.4| 

In [18]:
rawInput.printSchema()

root
 |-- sepal length: double (nullable = true)
 |-- sepal width: double (nullable = true)
 |-- petal length: double (nullable = true)
 |-- petal width: double (nullable = true)
 |-- class: string (nullable = true)



In [19]:
from pyspark.ml.feature import StringIndexer

In [20]:
stringIndexer = StringIndexer(inputCol="class", outputCol="classIndex")
model = stringIndexer.fit(rawInput)

In [21]:
labelTransformed = model.transform(rawInput).drop("class")

In [22]:
labelTransformed.show()

+------------+-----------+------------+-----------+----------+
|sepal length|sepal width|petal length|petal width|classIndex|
+------------+-----------+------------+-----------+----------+
|         5.1|        3.5|         1.4|        0.2|       0.0|
|         4.9|        3.0|         1.4|        0.2|       0.0|
|         4.7|        3.2|         1.3|        0.2|       0.0|
|         4.6|        3.1|         1.5|        0.2|       0.0|
|         5.0|        3.6|         1.4|        0.2|       0.0|
|         5.4|        3.9|         1.7|        0.4|       0.0|
|         4.6|        3.4|         1.4|        0.3|       0.0|
|         5.0|        3.4|         1.5|        0.2|       0.0|
|         4.4|        2.9|         1.4|        0.2|       0.0|
|         4.9|        3.1|         1.5|        0.1|       0.0|
|         5.4|        3.7|         1.5|        0.2|       0.0|
|         4.8|        3.4|         1.6|        0.2|       0.0|
|         4.8|        3.0|         1.4|        0.1|    

In [23]:
from pyspark.ml.feature import VectorAssembler

In [24]:
vectorAssembler = VectorAssembler(inputCols=["sepal length", "sepal width", "petal length", "petal width"],
                                  outputCol="features")

In [25]:
xgbInput = vectorAssembler.transform(labelTransformed)

In [26]:
xgbInput.show()

+------------+-----------+------------+-----------+----------+-----------------+
|sepal length|sepal width|petal length|petal width|classIndex|         features|
+------------+-----------+------------+-----------+----------+-----------------+
|         5.1|        3.5|         1.4|        0.2|       0.0|[5.1,3.5,1.4,0.2]|
|         4.9|        3.0|         1.4|        0.2|       0.0|[4.9,3.0,1.4,0.2]|
|         4.7|        3.2|         1.3|        0.2|       0.0|[4.7,3.2,1.3,0.2]|
|         4.6|        3.1|         1.5|        0.2|       0.0|[4.6,3.1,1.5,0.2]|
|         5.0|        3.6|         1.4|        0.2|       0.0|[5.0,3.6,1.4,0.2]|
|         5.4|        3.9|         1.7|        0.4|       0.0|[5.4,3.9,1.7,0.4]|
|         4.6|        3.4|         1.4|        0.3|       0.0|[4.6,3.4,1.4,0.3]|
|         5.0|        3.4|         1.5|        0.2|       0.0|[5.0,3.4,1.5,0.2]|
|         4.4|        2.9|         1.4|        0.2|       0.0|[4.4,2.9,1.4,0.2]|
|         4.9|        3.1|  

In [27]:
xgbInput = xgbInput.select("features", "classIndex")

In [28]:
train, test = xgbInput.randomSplit([0.8,0.2])

In [29]:
train.show()

+-----------------+----------+
|         features|classIndex|
+-----------------+----------+
|[4.3,3.0,1.1,0.1]|       0.0|
|[4.4,2.9,1.4,0.2]|       0.0|
|[4.4,3.0,1.3,0.2]|       0.0|
|[4.4,3.2,1.3,0.2]|       0.0|
|[4.5,2.3,1.3,0.3]|       0.0|
|[4.6,3.2,1.4,0.2]|       0.0|
|[4.6,3.4,1.4,0.3]|       0.0|
|[4.6,3.6,1.0,0.2]|       0.0|
|[4.8,3.0,1.4,0.3]|       0.0|
|[4.8,3.1,1.6,0.2]|       0.0|
|[4.8,3.4,1.6,0.2]|       0.0|
|[4.8,3.4,1.9,0.2]|       0.0|
|[4.9,2.4,3.3,1.0]|       1.0|
|[4.9,2.5,4.5,1.7]|       2.0|
|[4.9,3.0,1.4,0.2]|       0.0|
|[4.9,3.1,1.5,0.1]|       0.0|
|[4.9,3.1,1.5,0.1]|       0.0|
|[5.0,2.0,3.5,1.0]|       1.0|
|[5.0,2.3,3.3,1.0]|       1.0|
|[5.0,3.0,1.6,0.2]|       0.0|
+-----------------+----------+
only showing top 20 rows



load xgb module

In [30]:
import sparkxgb

In [31]:
xgb_model = sparkxgb.XGBoostClassifier(
    featuresCol="features",
    labelCol="classIndex"
)

In [32]:
xgb_model.setParams(eta=0.1,
                   maxDepth=2,
                   objective="multi:softprob",
                   numClass=3,
                   numRound=10,
                   numWorkers=2)

XGBoostClassifier_37636399a1dd

In [33]:
model = xgb_model.fit(train)

In [34]:
results = model.transform(test)

In [35]:
results.show(80)

+-----------------+----------+--------------------+--------------------+----------+
|         features|classIndex|       rawPrediction|         probability|prediction|
+-----------------+----------+--------------------+--------------------+----------+
|[4.6,3.1,1.5,0.2]|       0.0|[1.45413875579833...|[0.70844513177871...|       0.0|
|[4.7,3.2,1.3,0.2]|       0.0|[1.45413875579833...|[0.70844513177871...|       0.0|
|[4.7,3.2,1.6,0.2]|       0.0|[1.45413875579833...|[0.70844513177871...|       0.0|
|[4.8,3.0,1.4,0.1]|       0.0|[1.45413875579833...|[0.70844513177871...|       0.0|
|[4.9,3.1,1.5,0.1]|       0.0|[1.45413875579833...|[0.70844513177871...|       0.0|
|[5.0,3.4,1.5,0.2]|       0.0|[1.45413875579833...|[0.70844513177871...|       0.0|
|[5.0,3.6,1.4,0.2]|       0.0|[1.45413875579833...|[0.70844513177871...|       0.0|
|[5.1,3.7,1.5,0.4]|       0.0|[1.45413875579833...|[0.70844513177871...|       0.0|
|[5.4,3.0,4.5,1.5]|       1.0|[-0.1398261785507...|[0.15833075344562...|    

# Evaluate

In [36]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction",
                                              labelCol="classIndex",
                                              metricName="weightedPrecision")



perfect accuracy?

In [37]:
# compute the classification error on test data.
accuracy = evaluator.evaluate(results)
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.0316092


In [38]:
spark.stop()